# Bug prediction with LSTM Recurrent Neural Networks
In this notebook we explore the possiblity of utilizing a sequential model to detect logical bugs in source code. We are using a similar technique in Nautrual Language Processing (NLP) to sentiment analysis to analize a piece of code. The idea behind this approach is fairly simple, we look at a small snippet of code to determine whether the code is buggy or non-buggy.

This notebook will cover some machine learning topics such as recurrent neural networks and long short-term memory units (LSTMs). We also breifly discuss techniques for padding squences in tensorflow. 

# Word Embeddings for source code
In order to understand how deep leaning can be applied to analize source code, one popular technique is by creating word embeddings for each word in the code. To understand why we use word embeddings, we need to think about how neural networks work. For a neural network to run efficently the dot products of matrices are often used to make calculations. So, the network will be expecting scalar values, or a vector of scalar values, so it's obvious we will not be able to just feed string values into the model. We need a way to represent strings with scalar values. One approach can be to use one-hot representations to prepair the data, however this medthod doesn't give the model much information about what the word actually means.

For better results we will used word embeddings, which require a machine learning method to generate, namely using a skip-gram model. Put simiply Word2Vec will process the corpus, looking at one block of text at a time to train a neural network to guess the current word given some amount of previous words. But we do not need to worry about that as we are using a premade Word2Vec model provided by SciKit Learn. 

(The embeddings for this project are being generated in the json_to_vector.py file of this project.)

![bugAnalysis2.png](imgs/bugAnalysis2.png)

Now when we do this, each example we feed into the network our input is now a (number of words in the example) x D-dimensional matrix. The dimension of the embeddings we are currently using is 100 so the above example would be of size (16,100).

The vectors are generated in such a way that each dimension represents a feature and this vector will give the machine some context into what the word means and how it relates to other words.

![Figure_1000.png](imgs/Figure_1000.png)

You can see word that have a similar meaning are grouped together, in the example above we can see trig fuctions are grouped next to eachother as you would expect. Before we create these embeddings we are going to tokenize the source code and I will explain how and why we are doing this. The model is taking a large corpus of source code and outputs vectors for each unique word and token. We store the output into an embedding matrix.

![code2emb.png](imgs/code2emb.png)

# Tokenizing Source Code
In this project we offer a simple Java tokenizer that will replace any Java syntax with token. We do this for a few reasons, one reason is beacuse it helps split the code apart and we don't have to rely on whitespace to divide the words. For example, it ensures the line "area=5;" will split into 4 seperate values instead of generating 1 embedding for the entire line. Another reason we did this instead of removing all syntax is it will give the model more information about how words are related to eachother. The hope is the model will be able to more easily learn logic pattern, for example, when the model sees something like "foo(bar);" we would like the model to learn there is some relationship between foo and bar since bar is in parentheses directly after foo, i.e. we would like the model to more easily learn bar is an argument in a call to foo.

#### Photo placeholder

# Generating Buggy Datasets
In this project we also take a look at generating buggy datasets for training. With the source code tokenized we can search corpus's of code for basic logic patterns. Right now we are only generating bugs with swapped arguments. Meaning, we take a line of that has a function call with 2 arguments and simply swap the arugments so that we have an example of code that is likely working and a piece of code that is buggy.

#### Photo placeholder

We are only using examples of swapped arguments so that will be the bug this model will be able to detct. But this model can be scaled quite easily, all you would have to do is look into generating different logical bugs so the model can learn a wider variety of bugs. 

Another idea to help with generating bugs would be to implement some kind of tagging system, which was outside the scope of this project. To read more about tagging and why it would make generating bugs even easier refer to the Natural Language Toolkit's (NLTK) __[tagging documentation.](https://www.nltk.org/book/ch05.html)__
# Recurrent Neural Networks (RNNs)
Now that we have everything set up we are ready to jump into deep learning! In Natural Language Processing (NLP) there is a squential aspect to the data, meaning the order of the words is important. Similarily in code what comes before or after a word/syntax is very important. In order to keep track of the order of input we must utilize a squence model, a recurrent neural network.

The structure of a recurrent neural network is different from a simple feedfoward neural network. In a traditional NN input is taken in all at once, the input also has a fixed size. They look something like this...

![TraditionalNN.png](imgs/NN.png)

The main difference in a RNN is that we now take input in a squential fasion. Each word in the input is now associated with a specific time step. 

![RNN.png](imgs/RNN.png)

At each time step will calculate activation values that will be passed along to the next time step, often referred to as hidden states. These hidden states will contain information about what the model has already seen in previous time steps. At each step the hidden state is calculated using the current input, $x_{t}$, and the previsous hidden state, $h_{t-1}$. There is also a bias being added what was left out of this equations. The sigma below refers to an activation function which is normally tanh, and sometimes sigmoid.

![HiddenState.png](imgs/HiddenState.png)

The 2 W terms refer to the weight matrices. Note that the superscripts are different, it is a common notation standard to label the weight matrices with a superscript or subscript that informs you where to use that matrix. Moreover, for the $W^{X}$ matrix, the superscript X tells us this martix is assosiated with the input, detoned $x_{t}$, and we should take the dot product of the two. The same logic applies to the $W^{H}$ matrix. Both of the weight matrices are shared at each time step.

The weight matrices are updated through an optimization process called backpropagation through time.

At the last time step we take the dot product of the activation values and a third $W^{A}$ matrix and feed the value to a sigmoid function to get a descrete output, between 0 and 1, representing the how confident the model is the input was a buggy piece of code.

#### Photo placeholder

# Long Short-Term Memory Units (LSTMs)

![LSTM.png](imgs/LSTM.png)


In [1]:
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
from scipy.spatial.distance import cdist
from keras.models import Sequential
from keras.layers import Dense, CuDNNLSTM, Embedding
from keras.optimizers import RMSprop, Adam
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences

/home/paperspace/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
import json
with open("clean.json") as f:
    clean = json.load(f)
with open("buggy.json") as f:
    buggy = json.load(f)
with open("py2vec_modelJ.json") as f:
    embs = json.load(f)

In [3]:
clean = np.asarray(clean)
buggy = np.asarray(buggy)
buggy_labels = np.ones(len(buggy))
clean_labels = np.zeros(len(clean))

In [4]:
embedding_matrix = []
int_to_word = []
word_to_int = {}
i = 0
for word, emb in embs.items():
    embedding_matrix.append(emb)
    int_to_word.append(word)
    word_to_int[word] = i
    i += 1
    
embedding_matrix.append(np.zeros(100))
embedding_matrix = np.asarray(embedding_matrix)
print(word_to_int['safemax'])
print(int_to_word[2])
print(np.array_equal(embs['safemax'], embedding_matrix[2]))

2
safemax
True


In [5]:
train_data = np.concatenate((clean, buggy), axis=0)
train_labels = np.concatenate((clean_labels, buggy_labels), axis=0)

for i in range(train_data.shape[0]):
    string = ''
    for j in range(len(train_data[i])):
        string += train_data[i][j] + ' '
    train_data[i] = string
    
np.random.seed(3)
np.random.shuffle(train_data)
np.random.seed(3)
np.random.shuffle(train_labels)

In [6]:
test_data = train_data[train_data.shape[0]-1000:]
test_labels = train_labels[train_labels.shape[0]-1000:]
train_data = train_data[:train_data.shape[0]-1000]
train_labels = train_labels[:train_labels.shape[0]-1000]

num_words = len(embs)

In [7]:
train_data_tokens = []
test_data_tokens = []
num_words_missed = 0
num_words_found = 0
for i in range(train_data.shape[0]):
    train_data_tokens.append([])
    for word in train_data[i].split():
        if word.lower() in embs:
            train_data_tokens[i].append(word_to_int[word.lower()])
            num_words_found += 1
        else:
            train_data_tokens[i].append(-1)
            num_words_missed += 1
for i in range(test_data.shape[0]):
    test_data_tokens.append([])
    for word in test_data[i].split():
        if word.lower() in embs:
            test_data_tokens[i].append(word_to_int[word.lower()])
            num_words_found += 1
        else:
            train_data_tokens[i].append(embedding_matrix.shape[0]-1)
            num_words_missed += 1
print("Number of words embedding found %d" % num_words_found)
print("Number of words embedding missing %d" % num_words_missed)

Number of words embedding found 497424
Number of words embedding missing 13988


In [8]:
print(train_data_tokens[0])
print(test_data_tokens[0])
print(embedding_matrix.shape[0])

[400, 5404, 3580, 2298, 2561, 3877, 4603, 1649, 5626, 4029, 994, 5934]
[994, 435, 2561, 1728, 4603, 2170, 5626, 5626, 1937]
5935


In [10]:
print(train_data_tokens[2])
int_to_word.append("unknown")
def tokens_to_string(tokens):
    words = [int_to_word[token] for token in tokens if token != 0]
    text = " ".join(words)
    return text
print(tokens_to_string(train_data_tokens[2]))

[2526, 2529, 4068, 4068, 5340, 994, 3984, 2561, 2905, 4603, 5918, 5626, 5934]
_atsignsymbol_ gwtincompatible _divide_ _divide_ doublemath _dispatch_ roundtoint _openparen_ double _comma_ roundingmode _closeparen_ unknown


In [11]:
num_tokens = [len(tokens) for tokens in train_data_tokens + test_data_tokens]
num_tokens = np.asarray(num_tokens)

In [12]:
np.mean(num_tokens)

14.09547433989306

In [13]:
np.max(num_tokens)

54

In [14]:
max_tokens = np.mean(num_tokens) + 2 * np.std(num_tokens)
max_tokens = int(max_tokens)
max_tokens

27

In [15]:
np.sum(num_tokens < max_tokens) / len(num_tokens)

0.9382613968358966

In [16]:
max_tokens = np.max(num_tokens)

In [17]:
pad = 'pre'

In [18]:
train_data_pad = pad_sequences(train_data_tokens, maxlen=max_tokens,
                              padding=pad, truncating=pad)
test_data_pad = pad_sequences(test_data_tokens, maxlen=max_tokens,
                             padding=pad, truncating=pad)

In [19]:
train_data_pad.shape

(35282, 54)

In [20]:
np.array(train_data_pad[0])

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,  400, 5404,
       3580, 2298, 2561, 3877, 4603, 1649, 5626, 4029,  994, 5934],
      dtype=int32)

In [21]:
from keras.layers import Dropout
num_words = len(int_to_word)
model = Sequential()
model.add(Embedding(input_dim=embedding_matrix.shape[0],
                   output_dim=embedding_matrix.shape[1],
                   input_length=max_tokens,
                   weights=[embedding_matrix],
                   trainable=False,
                   name='embedding_layer'))
#model.add(Embedding(input_dim=num_words,
#                   output_dim=100,
#                   input_length=max_tokens,
#                   name='embedding_layer'))
model.add(CuDNNLSTM(16, return_sequences=True))
model.add(Dropout(0.2))
model.add(CuDNNLSTM(8))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
optimizer = Adam(lr=1e-3)
model.compile(loss='binary_crossentropy',
             optimizer=optimizer,
             metrics=['accuracy'])

Instructions for updating:
Use the retry module or similar alternatives.


In [22]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_layer (Embedding)  (None, 54, 100)           593500    
_________________________________________________________________
cu_dnnlstm_1 (CuDNNLSTM)     (None, 54, 16)            7552      
_________________________________________________________________
dropout_1 (Dropout)          (None, 54, 16)            0         
_________________________________________________________________
cu_dnnlstm_2 (CuDNNLSTM)     (None, 8)                 832       
_________________________________________________________________
dropout_2 (Dropout)          (None, 8)                 0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 9         
Total params: 601,893
Trainable params: 8,393
Non-trainable params: 593,500
_________________________________________________________________


In [23]:
%%time
model.fit(train_data_pad, train_labels,
         validation_split=0.05, epochs=10, batch_size=64)

Train on 33517 samples, validate on 1765 samples
Epoch 1/10
33517/33517 [==============================] - 8s 237us/step - loss: 0.6935 - acc: 0.5052 - val_loss: 0.6898 - val_acc: 0.5439
Epoch 2/10
33517/33517 [==============================] - 6s 168us/step - loss: 0.6155 - acc: 0.6462 - val_loss: 0.5089 - val_acc: 0.7275
Epoch 3/10
33517/33517 [==============================] - 6s 172us/step - loss: 0.4859 - acc: 0.7485 - val_loss: 0.4309 - val_acc: 0.7841
Epoch 4/10
33517/33517 [==============================] - 6s 170us/step - loss: 0.4358 - acc: 0.7776 - val_loss: 0.3932 - val_acc: 0.7977
Epoch 5/10
33517/33517 [==============================] - 6s 168us/step - loss: 0.4088 - acc: 0.7922 - val_loss: 0.3757 - val_acc: 0.8062
Epoch 6/10
33517/33517 [==============================] - 6s 166us/step - loss: 0.3913 - acc: 0.7984 - val_loss: 0.3649 - val_acc: 0.8125
Epoch 7/10
33517/33517 [==============================] - 6s 168us/step - loss: 0.3741 - acc: 0.8056 - val_loss: 0.3518 - v

In [24]:
result = model.evaluate(test_data_pad, test_labels)

1000/1000 [==============================] - 0s 152us/step


In [25]:
print("accuracy: {0:.2%}".format(result[1]))

accuracy: 80.10%
